In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [70]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [82]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f'**{other}')

        def _backward():
            self.grad += other * self.data ** (other - 1) * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * other ** -1

    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t ** 2) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

import random
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    def parameters(self): #calling it this way because pytorch has params on every single nn module. Does same thing. 
        #Returns the param tensors.
        #for us, return the param scalars
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
        
    def parameters(self):
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        return params
        # can also write it as return [p for neuron in self.neurons for p in neuron.parameters()

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [90]:
#have to reinit the MLP with the new functionality, then call n.parameters.
#now, we get all of the weights and biases in the network. 
n.parameters()
#and we can get total number of params in the MLP
len(n.parameters())

41

In [113]:
# Example dataset
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]

# Neural network
n = MLP(3, [4, 4, 1])

# Predictions
ypred = [n(x) for x in xs]

# Convert ys elements to Value objects
ys_value = [Value(y) for y in ys]

# Calculate the loss as a single Value object
loss = sum([(yout - ygt) ** 2 for ygt, yout in zip(ys_value, ypred)], Value(0.0))
loss

#look at the weight of the first neuron in the first layer
#n.layers[0].neurons[0].w[0]

Value(data=5.3627639612570315)

In [155]:
# Call backward on the loss
loss.backward() 

In [141]:
n.layers[0].neurons[0].w[0].data

0.7549340458749865

In [142]:
##something magical happened -> now... the value also has a grad because of the backward pass
n.layers[0].neurons[0].w[0].grad

#because the gradient in this particular weight in this particular neuron and particular layer is negative, it's influence on the loss
#is also negative (when I'm running it now it's negative). So increasing it makes the loss go down. Have this info for all neurons and params

1.9744494047819026

In [240]:
#now, we want to change all 41 params of the MLP slightly to lower the loss. When I'm running now, the grad 
#for n.layers[0].neurons[0].w[0] is pos, so I will negatively change the params to make loss go down
#tiny update in gradient descent scheme
#thinking of the gradient as a vector pointing in the direction of increased loss. We have to reverse it in the dir away from the loss
#modifying p.data by a small step size in the direction of the gradient
for p in n.parameters():
    p.data -= 0.005 * p.grad #data goes up

#slightly increasing the data makes the loss go down if the gradient is negative           loss<----d1---->d1 (loss goes down)
#sligtly decreasing the data makes the loss go up if the gradient is negative              loss<----d2<----d1 (loss goes up)

#slightly increasing the data mekes the loss go up if the graident is positive              --d1----->d2---->loss (loss goes up)
#sligtly decreasing the data makes the loss go down if the gradient is positive            ----d2<----d1---->loss (loss goes down)


#low loss means our predictions are matching the targets. Now, data is slightly closer to the targers.

#keep improving the loss with forward pass, backward pass, and update parameters (this block and next two. Then check ypred to validate
#that the predictions are close to the target values

#be careful not to get too cocky with the size of the step... we're only aware of the local bell curve, so we could 
#go up the other way. If you set the learning rate too low, you'll take way to long to converge. But if you go too high, 
#the whole thing gets unstable and you explode to the other side


In [241]:
#Recalculate the loss
ypred = [n(x) for x in xs]

# Calculate the loss as a single Value object
loss = sum([(yout - ygt) ** 2 for ygt, yout in zip(ys_value, ypred)], Value(0.0))
loss


Value(data=0.00010702998053131608)

In [242]:
# Call backward on the loss
loss.backward() 

In [243]:
ypred

[Value(data=0.99914010881412),
 Value(data=-0.9943214710049103),
 Value(data=-0.9914418797524301),
 Value(data=0.9991036440663317)]

In [244]:
n.parameters()
#this is the setting of weights and biases that makes our network predict our desired targets
#very close. So we implemented a neural net. Now, let's make it respectable and implement a training loop

[Value(data=1.0095700637038971),
 Value(data=-0.2377804510867051),
 Value(data=-0.6667434643261337),
 Value(data=0.3842807350753076),
 Value(data=-1.1958454644814924),
 Value(data=-0.8062319207594539),
 Value(data=0.308509936292178),
 Value(data=0.22821869087598673),
 Value(data=-0.054324477150777456),
 Value(data=-1.2667156541510765),
 Value(data=1.1273937307716906),
 Value(data=0.6080648588155663),
 Value(data=-0.30847622437818367),
 Value(data=-0.9443266665067251),
 Value(data=-0.8960665327582183),
 Value(data=-0.9460081448588619),
 Value(data=-0.5167834602655089),
 Value(data=-1.345041677298339),
 Value(data=-0.3713106306958373),
 Value(data=-0.6855116853333872),
 Value(data=1.1719433660479348),
 Value(data=-0.8645659973502634),
 Value(data=0.1573466064261112),
 Value(data=1.3637558674496009),
 Value(data=-0.008230600573384062),
 Value(data=0.5042778321760878),
 Value(data=0.08303922229823205),
 Value(data=0.3517083601731989),
 Value(data=0.8473349793586711),
 Value(data=-0.7154166

In [108]:
#draw_dot(loss) 
#previously we looked at the draw_dot for a single forward pass. 
#This one actually forwarded all four examples, loss on top with mean-squared error. Note the massive graph. Four forward passes of
#a neural net for everyone of the examples, with loss on top, and ends with the value of the loss. This loss now backprops through 
#all 4 forward passes and every single intermediate
#value of teh neural net, all the way back to the params of the weights (the inputs to the neural net)
# (self.w + b, and xs ^^)
#gradients for the inputs don't matter because input is fixed

In [ ]:
#Now, we are going to make some convenience code so we can operate on all of the parameters at once
#- def params func
